In [1]:
import numpy as np

## Further Study

### Stochastic Gradient Descent

The only difference is how we calculate the loss and gradient which is based on only **one sample**.

$$\frac{\partial J}{\partial \theta_j} = (h^{(i)}-y^{(i)})x_j$$

### Mini-Batch Gradient Descent

The only difference is how we calculate the loss and gradient which is based on only **subset of samples**.

$$\frac{\partial J}{\partial \theta_j} = \sum_{i=start}^{batch}(h^{(i)}-y^{(i)})x_j$$

where start is a randomized number within the range of $m$ and batch is a predefined batch size, typically around 100 to 500

### Closed Form

Gradient descent gives one possible mean for minimizing $J$, which uses iterative approach and may take time.  In the situation where we know that our cost function is strictly concave or convex, we can explicitly take its derivative to zero.  This process of such derivation is called obtaining the **normal equations** or **closed form**. 

The **closed form** of linear regression can be derived easily.  Let $\mathbf{X}$ be a matrix of shape $(m, n)$, $\boldsymbol{\theta}$ as shape $(n, )$, and $\mathbf{y}$ as vector of shape $(m, )$.  Instead of writing the cost function as power of square, we shall write it in matrix multiplication as follows:

$$\frac{\partial J}{\partial \boldsymbol{\theta}} (\mathbf{X}
 - \mathbf{y})^T*(\mathbf{X}
-\mathbf{y})$$

Recall the following properties:

$$\frac{\partial J}{\partial \mathbf{X}} \mathbf{X}^T\mathbf{X}=2\mathbf{X} \tag{A}$$
$$\frac{\partial J}{\partial \mathbf{X}} \mathbf{A}\mathbf{X}=\mathbf{A}^T$$
$$(\mathbf{X}\mathbf{y})^T = \mathbf{y}^T\mathbf{X}^T$$

Therefore

\begin{align*}
\frac{\partial J}{\partial \boldsymbol{\theta}} (\mathbf{X}\boldsymbol{\theta} - \mathbf{y})^T*(\mathbf{X}\boldsymbol{\theta}-\mathbf{y}) &= \frac{\partial J}{\partial \boldsymbol{\theta}} (\boldsymbol{\theta}^T\mathbf{X}^T\mathbf{X}\boldsymbol{\theta} - \boldsymbol{\theta}^T\mathbf{X}^T\mathbf{y} - \mathbf{y}^T\mathbf{X}\boldsymbol{\theta} + \mathbf{y}^T\mathbf{y})\\
&= 2\mathbf{X}^T\mathbf{X}\boldsymbol{\theta} - 2\mathbf{X}^T\mathbf{y} \tag{see note*}\\
&= \mathbf{X}^T\mathbf{X}\boldsymbol{\theta} - \mathbf{X}^T\mathbf{y} = 0 \tag{set derivative to 0}\\
&= \boldsymbol{\theta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}
\end{align*}


Note*: Since $\mathbf{X}\boldsymbol{\theta}$ is a vector, and so is $\mathbf{y}$, it doesn't matter what the order is, thus we can simply add them to 2.  Also, we got 2 in front of the first part because we have two $\theta$ (used the property A)


**Why not closed form always**.  The answer is simple.  It does not always exists or possible, for example, the cost function is not convex or concave.  But of course, if it exists, we usually prefer closed form given that it is usually faster than gradient descent.  Nevertheless, as you can see, taking inverse of huge number of features can be expensive, thus it is also not always straightforward thing to always prefer closed form.

Yes, that's it for most of the theoretical stuff.  Let's start implementing some of these concepts so we can better understand them.

The closed form is a normal equations derived from setting the derivatives = 0.  By performing only some inverse operations and matrix multiplication, we will be able to get the theta.

$$\boldsymbol{\theta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

When closed form is available, is doable (can be inversed - can use pseudoinverse), and with not many features (i.e., inverse can be slow), it is recommended to always use closed form.  

**Implementation steps:**

1. Prepare your data
    - add intercept
    - $\mathbf{X}$ and $\mathbf{y}$ and $\mathbf{w}$ in the right shape
        - $\mathbf{X}$ -> $(m, n)$
        - $\mathbf{y}$ -> $(m, )$
        - $\mathbf{w}$ -> $(n, )$
        - where $m$ is number of samples
        - where $n$ is number of features
    - train-test split
    - feature scale
    - clean out any missing data
    - (optional) feature engineering
2. Plug everything into the equation.  Here we shall use X_train to retrieve the $\boldsymbol{\theta}$
$$\boldsymbol{\theta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

3. We simply using the $\boldsymbol{\theta}$, we can perform a dot product with our X_test which will give us $\mathbf{\hat{y}}$.

4. We then calculate the errors using mean-squared-error function:

$$\frac{1}{m}\sum_{i=1}^m(h_\theta(x^{(i)}) - y^{(i)})^2$$

Note that it's a bit different from our $J(\boldsymbol{\theta})$ because $J(\boldsymbol{\theta})$ puts $\frac{1}{2}$ instead of $\frac{1}{m}$ for mathematical convenience for derivatives, since we know changing constants do not change the optimization results.


Let's implement.

### 1. Prepare data

#### 1.1 Get your X and y in the right shape for Sklearn

In [2]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()

In [3]:
X = diabetes.data
X.shape #number of samples, number of features

m = X.shape[0]  #number of samples
n = X.shape[1]  #number of features

In [4]:
y = diabetes.target

#### 1.2 Feature scale your data to reach faster convergence

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

#### 1.3 Train test split your data

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

assert len(X_train)  == len(y_train)
assert len(X_test) == len(y_test)

#### 1.4 Add intercepts

In [7]:
intercept = np.ones((X_train.shape[0], 1))

# concatenate the intercept based on axis=1
X_train = np.concatenate((intercept, X_train), axis=1)

# np.ones((shape))
intercept = np.ones((X_test.shape[0], 1))

# concatenate the intercept based on axis=1
X_test = np.concatenate((intercept, X_test), axis=1)

### Step 2: Fit your algorithm 

#### 1. Define your algorithm

In [8]:
from numpy.linalg import inv

# order of operation DOES NOT MATTER
# But don't flip y before X^T for example
def closed_form(X, y):
    return inv(X.T @ X) @ X.T @ y

In [9]:
# let's use the closed_form to find the theta
theta = closed_form(X_train, y_train)
theta  #<------this is our model

array([ 1.50394523e+02,  1.24951263e-01, -9.56655783e+00,  2.74079297e+01,
        1.38021711e+01, -2.82122671e+01,  1.24108051e+01,  1.50711994e+00,
        5.52299386e+00,  3.33037406e+01,  2.89952607e+00])

#### 2. Compute accuracy/loss

In [10]:
# Compute the accuracy/loss

yhat = X_test @ theta #==> X (m, n+1)  @ (n+1, ) w ==> (m, ) y

# if I want to compare yhat and y, I need to make sure they are the same shape
assert y_test.shape == yhat.shape

In [11]:
# get the mse
mse = ((y_test - yhat)**2).sum() / X_test.shape[0]
print("Mean squared errors: ", mse)

Mean squared errors:  2738.022740831692
